In [1]:
from typing_extensions import TypedDict

class State(TypedDict):
    topic: str
    draft: dict   # structured output
    feedback: str


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.1-8b-instant")

In [3]:
def collect_data(state: State):
    print("Collecting topic...")
    return {"topic": state["topic"], "draft": {}, "feedback": ""}


In [4]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# Schema
response_schemas = [
    ResponseSchema(name="title", description="Catchy title for the post"),
    ResponseSchema(name="intro", description="Introduction paragraph"),
    ResponseSchema(name="points", description="3-5 bullet points"),
    ResponseSchema(name="conclusion", description="Closing summary")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="Write a blog post about {topic}.\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": format_instructions},
)




In [5]:
def write_material(state: State):
    print("Writing draft...")
    topic = state["topic"]
    _input = prompt.format_prompt(topic=topic)
    response = llm.invoke(_input.to_string())

    # response is AIMessage → extract text
    parsed = output_parser.parse(response.content)
    return {"draft": parsed}


In [10]:
def review_material(state: State):
    print("Reviewing draft...")
    draft = state["draft"]
    feedback = state.get("feedback", "")

    if feedback:
        revised = llm.invoke(
            f"Rewrite this draft based on feedback: {feedback}\n\nDraft:\n{draft}"
        )
        parsed = output_parser.parse(revised.content)
        return {"draft": parsed}
    else:
        return {"draft": draft}


In [7]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("collect_data", collect_data)
builder.add_node("write_material", write_material)
builder.add_node("review_material", review_material)

builder.add_edge(START, "collect_data")
builder.add_edge("collect_data", "write_material")
builder.add_edge("write_material", "review_material")
builder.add_edge("review_material", END)

graph = builder.compile()


In [8]:
def display_post(draft: dict):
    print("\n" + "="*50)
    print("📌 Title:", draft["title"])
    print("\n📝 Intro:\n", draft["intro"])
    print("\n🔑 Key Points:")
    for p in draft["points"]:
        print("-", p)
    print("\n✅ Conclusion:\n", draft["conclusion"])
    print("="*50 + "\n")


In [11]:
while True:
    topic = input("Enter a topic for your post: ")
    if topic.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # First draft
    state = {"topic": topic, "draft": {}, "feedback": ""}
    result = graph.invoke(state)
    display_post(result["draft"])

    # Feedback stage
    feedback = input("Do you have feedback for revision? (leave empty if none): ")
    if feedback.strip():
        state["feedback"] = feedback
        result = graph.invoke(state)
        display_post(result["draft"])


Writing draft...
Reviewing draft...

📌 Title: The Haunting Tale of Ubume: Japan's Ghostly Bird Woman

📝 Intro:
 In Japanese folklore, there exist numerous ghostly creatures that are said to roam the land, bringing fear and superstition to those who cross their paths. Among these is the Ubume, a vengeful spirit that takes the form of a bird-like woman. With her haunting appearance and tragic backstory, the Ubume is a fascinating and terrifying figure in Japanese mythology.

🔑 Key Points:
- * The Ubume is said to be a vengeful spirit that appears as a woman with a bird-like face, often with a beak and wings.
- * According to legend, the Ubume was once a woman who died in childbirth, and her spirit was consumed by a deep sense of longing and loss.
- * In some stories, the Ubume is said to prey on the children of others, often taking them as a substitute for her own lost child.
- * The Ubume is often associated with the concept of 'tsukumogami,' or the spirit of an object that has formed a